In [154]:
import pandas as pd
import numpy as np
from typing import Union
import matplotlib.pyplot as plt
import seaborn as sns

In [155]:
df = pd.read_csv(r"C:\Users\ryj81\OneDrive\Desktop\sample for Ai project SC.csv", low_memory =False)
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45863 entries, 0 to 45862
Data columns (total 69 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   SAP ON                      45863 non-null  object 
 1   SAP item #                  45863 non-null  int64  
 2   SAP HLI                     45863 non-null  int64  
 3   Product                     45863 non-null  object 
 4   Product description         45863 non-null  object 
 5   Order type                  45863 non-null  object 
 6   Country                     45863 non-null  object 
 7   SYSTEM_ID                   45863 non-null  object 
 8   Factory Plant               45863 non-null  object 
 9   Plant                       45863 non-null  object 
 10  ConsolidationPlant          45863 non-null  object 
 11  Item status                 45863 non-null  object 
 12  Header status               45863 non-null  object 
 13  Order create dt             458

In [156]:
import re

def dedupe_columns(cols):
    seen = {}
    out = []
    for c in cols:
        if c not in seen:
            seen[c] = 0
            out.append(c)
        else:
            seen[c] += 1
            out.append(f"{c}_{seen[c]}")
    return out

def data_cleanse(df):
    df = df.copy()

    # 1) Clean column names FIRST
    df.columns = [
        re.sub(
            r"[^a-z0-9_]",
            "",
            str(c).strip().lower().replace(" ", "_")
        )
        for c in df.columns
    ]

    # 2) Deduplicate AFTER cleaning
    df.columns = dedupe_columns(df.columns)

    # 3) Clean string values safely
    obj_cols = df.select_dtypes(include=["object"]).columns
    for col in obj_cols:
        df[col] = df[col].astype(str).str.strip().str.lower()

    return df

df = data_cleanse(df)
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45863 entries, 0 to 45862
Data columns (total 69 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   sap_on                      45863 non-null  object 
 1   sap_item_                   45863 non-null  int64  
 2   sap_hli                     45863 non-null  int64  
 3   product                     45863 non-null  object 
 4   product_description         45863 non-null  object 
 5   order_type                  45863 non-null  object 
 6   country                     45863 non-null  object 
 7   system_id                   45863 non-null  object 
 8   factory_plant               45863 non-null  object 
 9   plant                       45863 non-null  object 
 10  consolidationplant          45863 non-null  object 
 11  item_status                 45863 non-null  object 
 12  header_status               45863 non-null  object 
 13  order_create_dt             458

In [157]:
#convert to datetime
date_cols = [
    "ordercreatedt",
    "crd",
    "esd",
    "pcsd",
    "hpreceivedt",
    "todayutc"
]

for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

df = data_cleanse(df)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45863 entries, 0 to 45862
Data columns (total 69 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   sap_on                      45863 non-null  object        
 1   sap_item_                   45863 non-null  int64         
 2   sap_hli                     45863 non-null  int64         
 3   product                     45863 non-null  object        
 4   product_description         45863 non-null  object        
 5   order_type                  45863 non-null  object        
 6   country                     45863 non-null  object        
 7   system_id                   45863 non-null  object        
 8   factory_plant               45863 non-null  object        
 9   plant                       45863 non-null  object        
 10  consolidationplant          45863 non-null  object        
 11  item_status                 45863 non-null  object    

In [158]:
def summary_stats(df) -> pd.DataFrame:

    # Create base frame
    stats = pd.DataFrame(index=df.columns)

    stats["dtype"] = df.dtypes


    stats["sample_val"] = [df[col].dropna().sample(1).iloc[0]
                            if df[col].notna().any() else np.nan
                            for col in df.columns]

    stats["vals"] = df.count()
    stats["miss_pct"] = (df.isna().sum() / len(df) * 100).round(1)
    stats["unique"] = df.nunique()

    # Initiate Descriptive Stats
    for col in ["mean", "mode", "min", "max", "std", "skew", "kurtosis"]:
        stats[col] = np.nan

    # Compute stats for numeric columns
    num_cols = df.select_dtypes(include="number").columns

    for col in num_cols:
        s = df[col].dropna()
        if not s.empty:
            stats.at[col, "mean"] = s.mean()
            stats.at[col, "min"] = s.min()
            stats.at[col, "max"] = s.max()
            stats.at[col, "std"] = s.std()
            stats.at[col, "skew"] = s.skew().round(1)
            stats.at[col, "kurtosis"] = s.kurtosis().round(1)
            mode_vals = s.mode()
            stats.at[col, "mode"] = mode_vals.iloc[0] if not mode_vals.empty else np.nan


    round_cols = ["mean", "mode", "min", "max", "std"]
    stats[round_cols] = stats[round_cols].round(2)

    return stats
stats = summary_stats(df)
stats.head(10)

,dtype,sample_val,vals,miss_pct,unique,mean,mode,min,max,std,skew,kurtosis
sap_on,object,7050021066,45863,0.0,5802,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sap_item_,int64,250,45863,0.0,455,438.68,10.0,10.0,5390.0,542.72,3.0,12.7
sap_hli,int64,10,45863,0.0,379,299.06,10.0,0.0,5360.0,498.63,3.4,16.2
product,object,p48908-b21,45863,0.0,5072,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_description,object,hpe ilo adv 1-svr lic 3yr support,45863,0.0,4121,NaN,NaN,NaN,NaN,NaN,NaN,NaN
order_type,object,zfd,45863,0.0,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
country,object,canada,45863,0.0,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
system_id,object,s4,45863,0.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
factory_plant,object,,45863,0.0,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
plant,object,8p2m,45863,0.0,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
def duplicate_stats(df) -> pd.DataFrame:
    stats = pd.DataFrame(index=df.columns)

    stats["total_rows"] = len(df)
    stats["dup_count"] = [df[col].duplicated(keep=False).sum() for col in df.columns]
    stats["dup_pct"] = (stats["dup_count"] / len(df) * 100).round(2)
    stats["unique"] = df.nunique()

    return stats  # Only show columns with duplicates

dup_stats = duplicate_stats(df)
print(dup_stats)
print(duplicate_stats(df))

                            total_rows  dup_count  dup_pct  unique
sap_on                           45863      44406    96.82    5802
sap_item_                        45863      45810    99.88     455
sap_hli                          45863      45805    99.87     379
product                          45863      43862    95.64    5072
product_description              45863      44432    96.88    4121
...                                ...        ...      ...     ...
ageing_receive_to_today_cd       45863      45760    99.78     571
hubreceivedt                     45863      45851    99.97      67
profit_center_l1_name            45863      45863   100.00       3
todayutc                         45863      45863   100.00       1
division                         45863      45863   100.00      22

[69 rows x 4 columns]
                            total_rows  dup_count  dup_pct  unique
sap_on                           45863      44406    96.82    5802
sap_item_                        45863 

In [174]:

#Feature engineering at LINE LEVEL


#aliases for repeated columns after cleansing 
import numpy as np
import pandas as pd

def find_col(df, must_contain):
    if isinstance(must_contain, str):
        must_contain = [must_contain]
    for c in df.columns:
        if all(token in c for token in must_contain):
            return c
    return None

# --- find columns (after cleaning!) ---
order_create_col = find_col(df, ["order", "create"])   # order_create_dt
crd_col  = find_col(df, "crd")
esd_col  = find_col(df, "esd")
pcsd_col = find_col(df, "pcsd")

hold_col = find_col(df, ["hold", "desc"])              # hold_desc
prio_col = find_col(df, ["del", "prio"])               # del_prio
comp_col = find_col(df, "complexity")                  # complexity

print(order_create_col, crd_col, esd_col, pcsd_col, hold_col, prio_col, comp_col)

# --- convert dates safely ---
for c in [order_create_col, crd_col, esd_col, pcsd_col]:
    if c is not None:
        df[c] = pd.to_datetime(df[c], errors="coerce")

# --- time deltas ---
df["order_to_crd_days"] = (df[crd_col] - df[order_create_col]).dt.days if crd_col and order_create_col else np.nan
df["crd_to_esd_days"]   = (df[esd_col] - df[crd_col]).dt.days if esd_col and crd_col else np.nan
df["esd_to_pcsd_days"]  = (df[pcsd_col] - df[esd_col]).dt.days if pcsd_col and esd_col else np.nan

# --- risk flags ---
df["has_hold"]      = df[hold_col].notna().astype(int) if hold_col else 0
df["high_priority"] = df[prio_col].isin(["1", "high", "urgent"]).astype(int) if prio_col else 0
df["complex_flag"]  = df[comp_col].isin(["high", "very high", "very_high"]).astype(int) if comp_col else 0



order_create_dt crd esd pcsd hold_desc del_prio complexity


In [170]:

#modeling by order level and not by line item level
#order_df = (
#    df
#    .groupby("sapon", as_index=False)
#    .agg({
        "ageingcreatetotoday": "max",
        "sumegnetusdamount": "sum",
        "sumqtyordered": "sum",
        "crd_to_esd_days": "max",
        "esd_to_pcsd_days": "max",
        "order_to_crd_days": "max",
        "has_hold": "max",
        "high_priority": "max",
        "complex_flag": "max",
        "sapitem#": "nunique"
 #   })
#    .rename(columns={"sapitem#": "n_items"}))


IndentationError: unexpected indent (869289120.py, line 6)